In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# Reads data
features = pd.read_csv("data/features.csv")
target = pd.read_csv("data/target.csv")

In [3]:
# Bins target values
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=4, encode = "onehot-dense", strategy = "quantile")

target_discretized = discretizer.fit_transform(target["G3"].values.reshape(-1, 1))
target_new = []
for row in target_discretized:
    target_new.append(list(row).index(1))

In [4]:
# Splits data into training, testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state = 1001)

In [5]:
# Gets final grade for target
t_train = y_train["G3"]
t_test = y_test["G3"]

In [6]:
# Imports Ridge and accuracy metrics
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs Ridge Regression
model = Ridge().fit(X=X_train, y=t_train)

# Measures accuracy
print("R-squared value for training set: ", r2_score(t_train, model.predict(X_train)))
print("R-squared value for testing set: ", r2_score(t_test, model.predict(X_test)))

R-squared value for training set:  0.34042660318538653
R-squared value for testing set:  0.22771777942292692


In [7]:
# Splits data for classification into training, testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_new, random_state = 1001)

In [43]:
# Imports SVC and accuracy metrics
from sklearn.svm import SVC
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Recommended using kernel

# Performs SVC
model = SVC(gamma = "scale").fit(X=X_train, y=y_train)

# Measures accuracy
accuracy_train = model.score(X_train, y_train)
accuracy_test = model.score(X_test, y_test)
print("Prediction accuracy on the train data:", f"{accuracy_train:.2%}")
print("Prediction accuracy on the test data:", f"{accuracy_test:.2%}")

Prediction accuracy on the train data: 37.42%
Prediction accuracy on the test data: 33.72%


In [8]:
# Imports Decision Tree and accuracy metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Performs Decision Tree Classifier
model = DecisionTreeClassifier().fit(X=X_train, y=y_train)

# Recommended using kernel -> we want to be able to model nonlinear things!
# Want to use feature expansion to classify them
# To choose the kernel: brute force/ try out a bunch of different ones (look at data and see if there's anything interesting you can pick up on, for example see if there's any 2 or more things that you can take the product of and it seems interesting)
# kernel: considering n features at the same time

# Measures accuracy
accuracy_train = model.score(X_train, y_train)
accuracy_test = model.score(X_test, y_test)
print("Prediction accuracy on the train data:", f"{accuracy_train:.2%}")
print("Prediction accuracy on the test data:", f"{accuracy_test:.2%}")

Prediction accuracy on the train data: 100.00%
Prediction accuracy on the test data: 36.78%


In [21]:
import torch

class NeuralNetwork(torch.nn.Module):
    def __init__(self, inputDim, outputDim, layerDim):
        
        super(NeuralNetwork, self).__init__()
        self.inputDim = inputDim
        self.outputDim = outputDim
        self.layerDim = layerDim


        self.l1 = torch.nn.Linear(self.inputDim, self.layerDim)
        self.relu = torch.nn.ReLU()
        self.l2 = torch.nn.Linear(self.layerDim, self.outputDim)
        self.relu = torch.nn.ReLU()

        
    def forward(self, x):
        hidden = self.l1(x)
        relu = self.relu(hidden)
        output = self.l2(relu)
        # output = self.relu(output)
        return output



In [9]:
X_test.iloc[3:6].values.shape

(3, 96)

In [22]:
model = NeuralNetwork(96, 20, 48)
for param in model.parameters():
    print(param.data)

tensor([[ 0.0004,  0.0139,  0.1010,  ...,  0.0185, -0.0828, -0.0452],
        [ 0.0258,  0.0610,  0.0891,  ..., -0.0968, -0.0176,  0.0666],
        [-0.0256,  0.0065,  0.0219,  ..., -0.0348,  0.0105,  0.0557],
        ...,
        [-0.0786, -0.0832,  0.0057,  ..., -0.0413,  0.0473, -0.0938],
        [-0.0646, -0.0600, -0.0633,  ..., -0.0691, -0.0881, -0.0751],
        [ 0.0054,  0.0450,  0.0137,  ..., -0.0528, -0.0843, -0.0382]])
tensor([ 0.0996, -0.0487,  0.0365,  0.0181,  0.0419, -0.0531, -0.0529, -0.0499,
         0.0630,  0.0954,  0.0784,  0.0680,  0.0020,  0.0160, -0.0736,  0.0307,
        -0.0902, -0.0586,  0.0978, -0.0893,  0.0395,  0.0879, -0.0726, -0.0165,
        -0.0488, -0.0735,  0.0473, -0.0660,  0.0933,  0.0236, -0.0434, -0.0891,
         0.0850, -0.0960,  0.0679,  0.0499,  0.0247,  0.0092,  0.0162, -0.0167,
         0.0099,  0.0418,  0.0605, -0.0702, -0.0654, -0.0005, -0.0111,  0.0303])
tensor([[ 1.7633e-03, -5.5875e-02, -3.6361e-03,  5.1476e-02,  4.6694e-02,
          1

In [13]:
torch.tensor(t_train.values[0]).reshape(-1).shape

torch.Size([1])

In [14]:
x_test_tensor = torch.FloatTensor(X_test.iloc[3:5].values)
y_pred = model(x_test_tensor)
y_pred.shape

torch.Size([2, 20])

In [37]:
# Now have to loop through all my data points to optimize my model - for loop looping through groups of (5) data points at a time using indexing

criterion = torch.nn.CrossEntropyLoss()
loss = criterion(y_pred, torch.tensor(t_train.values[0:2]).reshape(-1))

# model.eval()
# x_test_tensor = torch.FloatTensor(X_test.iloc[3].values[1:])
# y_pred = model(x_test_tensor)
# before_train = criterion(y_pred.squeeze(), torch.FloatTensor(t_test.values[0]))
# print('Test loss before training' , before_train.item())

# training mode vs. evaluation mode
# different architecture @ training/eval times
# model.train()
# y_pred = model(torch.FloatTensor(X_train.iloc[3].values[1:]))
# y_pred

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# 2 inputs, first is model.parameters() - includes all parameters in the model you want to optimize, make sure model.parameters() includes everything you want to optimize

for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    x_train_tensor = torch.FloatTensor(X_train.values)
    y_pred = model(x_train_tensor)

    # Compute and print loss.
    loss = criterion(y_pred, torch.tensor(t_train.values).reshape(-1))
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

99 6.123427867889404
199 3.3155951499938965
299 2.5472023487091064
399 2.4920718669891357
499 2.456698417663574


In [40]:
x_test_tensor = torch.FloatTensor(X_test.values)
test_results = model.forward(x_test_tensor)

In [41]:
for t in test_results:
    print(t)
    print(" ")

06, -30.0414, -47.0334,  -6.1454,  -5.1790,
         -4.6003,  -3.7558,  -3.8424,  -2.4967,  -2.8050,  -3.2492,  -3.2053,
         -3.2211,  -3.3885,  -3.7393,  -3.9647,  -4.2647,  -5.7341],
       grad_fn=<SelectBackward>)
 
tensor([ -0.7600,  -2.6097, -12.1197,  -9.5193, -14.6477,  -1.9469,  -1.3182,
         -2.3879,  -1.4975,  -1.0218,  -0.3144,  -0.4553,  -0.9143,  -1.1001,
         -0.6654,  -0.8859,  -1.1131,  -1.5938,  -2.1520,  -3.0025],
       grad_fn=<SelectBackward>)
 
tensor([ -1.5967,  -4.1463, -18.6346, -14.5676, -22.6482,  -2.9931,  -2.3920,
         -3.0865,  -2.0529,  -1.7341,  -0.6930,  -0.7871,  -1.4074,  -1.7213,
         -1.5387,  -1.5824,  -1.8977,  -2.2164,  -2.7960,  -3.6900],
       grad_fn=<SelectBackward>)
 
tensor([ -3.6643,  -7.8569, -35.4357, -27.8251, -43.6609,  -5.6383,  -5.1226,
         -4.2853,  -3.5308,  -3.5479,  -2.7160,  -2.4181,  -2.9719,  -3.0295,
         -2.9479,  -3.3496,  -3.4100,  -3.3953,  -4.2742,  -5.3172],
       grad_fn=<SelectBackwar

In [26]:
model.forward(x_test_tensor)[1]

tensor(35)

In [32]:
t_test

848      8
529     12
348     15
805     17
45       6
377     10
570      8
226     15
441     13
457     10
477     11
149     10
190     13
907      7
401     13
251     10
581     10
419     10
632     10
594     14
734     15
542     14
899     13
780     10
514     14
787     15
361     12
880     11
799     16
20      15
        ..
692     10
344     10
367      0
216      4
42      18
469     11
186     11
431     14
827      7
247      8
876     11
287     12
863     13
255      8
75      10
593     16
522     11
588     13
209      7
858     12
1008    10
537     12
284     11
494     13
640     13
356     13
936     11
724     14
962      0
779     10
Name: G3, Length: 261, dtype: int64